# Black Friday Plan

## To-do
* Index match retailer_dict to merged_data

## Requirements
* Total web traffic per sector at daily granularity
* YoY web traffic per sector at daily granularity
* Top 20 most popular websites on a weekly granularity
    * Include previous year position
* November and December
* Pull data every Weds covering the previous Sunday to Saturday

## Dates

| Reporting period | Deliver on |
|:-----------:|:----------:|
|24th October-30th October|3rd November|
|31st October-6th November|10th November|
|7th November-13th November|17th November|
|14th November-20th November|24th November|
|21st November-27th November|1st December|
|28th November-4th December|8th December|

In [1]:
from IPython.display import JSON
import datetime
import os
import sys
import requests
import json
import pandas as pd
import xlsxwriter as xl

In [2]:
# OPEN API KEY
with open('key', 'r') as key: 
    key = key.read()

# OPEN URL LIST
with open('urls', 'r') as urls:
    urls = [x.strip() for x in urls.readlines()]

# PRINT COST OF TOTAL DOWNLOAD
print(f'Downloading will cost {len(urls)} API hits')

In [9]:
# LAST 28 DAYS TRAFFIC FOR DOMAIN IN 'URLS' (COST = 1 API HIT PER DOMAIN)
for url in urls:
     if os.path.exists(f'2021_json/{url}.json'):
          pass
     else:
          with open(f"2021_json/{url}.json", "w") as outfile:
               json.dump(requests.get(
               f"https://api.similarweb.com/v1/website/"
               f"{url}/total-traffic-and-engagement/visits?api_key="
               f"{key}&country=gb&granularity=daily&"
               f"main_domain_only=false&format=json"   
               ).json(), outfile, indent=4)

In [10]:
# DAILY TRAFFIC FOR DOMAIN IN 'URLS' (COST = 1 API HIT PER DOMAIN)
for url in urls:
     if os.path.exists(f'2020_json/{url}.json'):
          pass
     else:
          with open(f"2020_json/{url}.json", "w") as outfile:
               json.dump(requests.get(
               f"https://api.similarweb.com/v1/website/"
               f"{url}/total-traffic-and-engagement/visits?api_key="
               f"{key}&start_date=2020-11&end_date=2020-12&"
               f"country=gb&granularity=daily&"
               f"main_domain_only=false&format=json"   
               ).json(), outfile, indent=4)

In [11]:
# CREATE VARIABLES FOR CSV PROCESSING
remove = ['country', 'granularity', 'mtd', 'main_domain_only',
          'show_verified', 'state', 'page', 'format', 
          'start_date', 'end_date']
merged_df = []

# OPEN JSON AND APPEND TO DATAFRAME
for url in urls:
    with open(f"2021_json/{url}.json", "r") as infile:
        df = json.load(infile)
        df = pd.json_normalize(df, 'visits').assign(**df["meta"]["request"])
        df = df.drop(columns=remove)
        merged_df.append(df)

# FORMAT DATAFRAME
merged_df = pd.concat(merged_df)
merged_df = merged_df.reset_index(drop=True)
merged_df['date'] = pd.to_datetime(merged_df['date'])
merged_df = merged_df.set_index('date')
merged_df = merged_df.loc['2021-10-31':'2021-11-06'] # UPDATE WEEKLY

In [12]:
# CREATE DATAFRAME FOR SECTOR INFO
retailer_info = pd.read_csv('retailer_dict.csv')
retailer_info = retailer_info.drop(columns='Retailer Name')
retailer_info = retailer_info.rename(columns={"Website": "domain", "Category": "sector"})
retailer_info = retailer_info.set_index('domain')

In [13]:
# JOIN DATAFRAMES
merged_df = merged_df.reset_index(drop=False)
merged_df = merged_df.set_index('domain')
merged_df = merged_df.join(retailer_info)
merged_df = merged_df.reset_index()
merged_df = merged_df.set_index('date')
merged_df.to_csv('2021_json/merged_data.csv', encoding='utf-8', index=True)

In [14]:
# SAVE DATAFRAMES TO CSV
domain_df = merged_df.pivot(columns='domain',values=['visits'])
domain_df = domain_df['visits']
domain_df = domain_df.reset_index()
domain_df = domain_df.set_index('date')
domain_df.to_csv('2021_json/domain_data.csv',encoding='utf-8',index=True)
sector_df = pd.pivot_table(merged_df,index='date',columns='sector',values='visits') 
sector_df.to_csv('2021_json/sector_data.csv',encoding='utf-8',index=True)

In [15]:
# DAILY TRAFFIC FOR 2020 TO CSV
merged_df = []

for url in urls:
    with open(f"2020_json/{url}.json", "r") as infile:
        df = json.load(infile)
        df = pd.json_normalize(df, 'visits').assign(**df["meta"]["request"])
        df = df.drop(columns=remove)
        merged_df.append(df)

merged_df = pd.concat(merged_df)
merged_df = merged_df.reset_index(drop=True)
merged_df['date'] = pd.to_datetime(merged_df['date'])
merged_df = merged_df.set_index('date')
merged_df = merged_df.loc['2020-11-01':'2020-12-31']
merged_df = merged_df.reset_index(drop=False)
merged_df = merged_df.set_index('domain')
merged_df = merged_df.join(retailer_info)
merged_df = merged_df.reset_index()
merged_df = merged_df.set_index('date')
merged_df.to_csv('2020_json/merged_2020.csv', encoding='utf-8', index=True)
domain_df = merged_df.pivot(columns='domain',values=['visits'])
domain_df = domain_df['visits']
domain_df = domain_df.reset_index()
domain_df = domain_df.set_index('date')
domain_df.to_csv('2020_json/domain_2020.csv',encoding='utf-8',index=True)
sector_df = pd.pivot_table(merged_df,index='date',columns='sector',values='visits') 
sector_df.to_csv('2020_json/sector_2020.csv',encoding='utf-8',index=True)

In [ ]:
# TEST LAST 28 DAYS TRAFFIC TO JSON (COST = 1 API HIT)
url = (f"https://api.similarweb.com/v1/website/bbc.com/"
       f"total-traffic-and-engagement/visits?api_key="
       f"{key}&country=world&granularity=daily&"
       f"main_domain_only=false&format=json")

response = requests.request("GET", url, headers={}, data={})

with open(f"{datetime.utcnow()}.json", "w") as outfile:
    json.dump(response.json(), outfile, indent=4)

JSON(response.json())